# Image Processing Class

In [1]:
class ImageProcessing:
    def __init__(self):
        self.using = True
        
    def image_thresholder1(self,stream): #different convolutions on blur
        ycrcb = cv2.cvtColor(stream, cv2.COLOR_BGR2YCrCb)
        blur = cv2.GaussianBlur(ycrcb,(5,5),0) #1,3,5,15,17,51,85,255
        thresh = cv2.inRange(blur, np.array([0, 135, 85]), np.array([255, 180, 135])) #pixelated
        return thresh    
    def image_thresholder2(self,stream): #different convolutions on blur
        ycrcb = cv2.cvtColor(stream, cv2.COLOR_BGR2YCrCb)
        blur = cv2.GaussianBlur(ycrcb,(15,15),0) #1,3,5,15,17,51,85,255
        thresh = cv2.inRange(blur, np.array([0, 135, 85]), np.array([255, 180, 135])) #pixelated
        return thresh
    def image_thresholder3(self,stream):
        gray = cv2.cvtColor(stream, cv2.COLOR_BGR2GRAY)
        ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY) 
        return thresh
    def image_thresholder4(self,stream):
        gray = cv2.cvtColor(stream, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray,(5,5),0)
        ret,thresh = cv2.threshold(blur,127,255,cv2.THRESH_BINARY)
        return thresh
    def image_thresholder5(self,stream): ## might be too detailed 
        gray = cv2.cvtColor(stream, cv2.COLOR_BGR2GRAY)
        ret,thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        return thresh
    def image_thresholder6(self,stream): 
        ycrcb = cv2.cvtColor(stream, cv2.COLOR_BGR2YCrCb)
        gray =  cv2.cvtColor(ycrcb, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray,(5,5),0) #1,3,5,15,17,51,85,255
        ret,thresh = cv2.threshold(blur,127,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        return thresh    
    def image_thresholder7(self, stream):
        ycrcb = cv2.cvtColor(stream, cv2.COLOR_BGR2YCrCb)
        gray =  cv2.cvtColor(ycrcb, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray,(5,5),0) #1,3,5,15,17,51,85,255
        ret,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        return thresh
    def image_thresholder8(self, stream):
        ycrcb = cv2.cvtColor(stream, cv2.COLOR_BGR2YCrCb)
        gray =  cv2.cvtColor(ycrcb, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray,(15,15),0) #1,3,5,15,17,51,85,255
        ret,thresh = cv2.threshold(blur,127,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        return thresh
    
    def CNN_resizer(self, stream, res_w, res_h):
        resized = imutils.resize(stream, width=res_w, height=res_h)
        return resized
    
    # ----------------------------------------

    def set_roi(self,frame, start_x,start_y, end_x,end_y):
        roi = frame[start_x:end_x, start_y:end_y]
        return roi#, cv2.rectangle(frame, (start_x,start_y),(end_x,end_y), (0,0,255),0)   


    def draw_roi(self, frame, start_x,start_y, end_x,end_y):
        roi = frame[start_x:end_x, start_y:end_y]
        return roi, cv2.rectangle(frame, (start_x,start_y),(end_x,end_y), (0,0,255),0)   

    #------------------------------------------


    def get_fps(self,device):
        fps = device.get(cv2.CAP_PROP_FPS)
        return fps


    def show_fps(self,frame, fps): #####REMOVE THIS ONE
        col = (0,0,255) #rgb
        pos = (10,20)
        font = cv2.FONT_HERSHEY_COMPLEX_SMALL
        scale = 1
        thick = 1
        return cv2.putText(frame, str(fps), pos, font, scale, col, thick)


    def put_text(self,frame, text, pos):
        col = (0,0,255) #rgb
        font = cv2.FONT_HERSHEY_COMPLEX_SMALL
        scale = 1
        thick = 1
        return cv2.putText(frame, str(text), pos, font, scale, col, thick)

    #------------------------------------------ 
    def get_contours(self,stream): 
        contours, hierarchy = cv2.findContours(stream, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        return contours


    def use_contours(self,stream):  
        cnts = self.get_contours(stream) 
        #cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        big_contour = max(cnts, key=cv2.contourArea) #theres a bug with max on an empty list

        # draw filled contour on black background
        filled = np.zeros_like(stream)
        cv2.drawContours(filled, [big_contour], -1, (255), cv2.FILLED)

        # get distance transform
        result = filled.copy()
        result = cv2.distanceTransform(result, distanceType=cv2.DIST_L2, maskSize=3, dstType=cv2.CV_8U)

        return result
    
    def get_hand_centre(self, stream):
        cnts = cv2.findContours(stream, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        for c in cnts: #can get float division by 0    
            M = cv2.moments(c)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
            else:
                cX, cY = 0, 0
        return cX, cY

    #------------------------------------------    
    def get_image_centre(self,frame):
        (h, w) = frame.shape[:2]
        return h, w

    def draw_guidelines(self,frame, r1, r2, r3, r4, h, w):
        cv2.circle(frame,(w//2,h//2), r1, (0,0,255), 2)
        cv2.circle(frame,(w//2,h//2), r2, (0,255,0), 2)
        cv2.circle(frame,(w//2,h//2), r3, (255,0,255), 2)
        cv2.circle(frame,(w//2,h//2), r4, (255,0,0), 2)

# GUI Class

In [10]:
from tkinter import Tk, ttk, Label, Canvas, Frame, Button, messagebox
import tkinter as tk
import numpy as np
import cv2
import PIL
from PIL import Image, ImageTk
import imutils
import tensorflow as tf
import datetime
import time
import threading
import os
import random


class Hand_GUI: 
    
    def __init__(self, master):
        self.master = master
        master.geometry('1400x1000+0+0')
        master.wm_title("HAND GUI")
        self.capture = cv2.VideoCapture(0)
        self.set_res(self.capture, 1280, 720)
        self.ret, self.frame = self.capture.read()
        self.imPRO = ImageProcessing()
        self.class_names = ['rock', 'call_me', 'rock_on', 'fingers_crossed', 'peace', 'paper', 'okay', 'up', 'scissor', 'thumbs']
        
        self.home_frame = True
        self.th_frame = False
        self.ha_frame = False
        self.class_frame = False
        
        self.new_data_frame = False
        self.add_central = False
        self.add_left = False
        self.add_right = False
        
        self.path = os.getcwd()
        
        
        self.selected_class = self.class_names[0]
        
        self.threads = []
        self.model =  tf.keras.models.load_model('MSc Thesistestingmodel3.hdf5') 
        
        self.create_tabs(master)
    
    def set_res(self, cap, x,y):
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, x)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, y)

        
# ------------ TABS -----------------------------------------------------------------------------------------------
    def create_tabs(self,container):
        tabs = ttk.Notebook(container)
        tabs.pack()
        
        go_live_frame = self.go_live_frame(container)
        thresh_frame = self.thresh_frame(container)
        cnn_frame = self.cnn_frame(container)
        hand_frame = self.hand_frame(container)
        new_data_frame = self.add_data_frame(container)
     
        go_live_frame.pack(fill="both", expand=1)
        cnn_frame.pack(fill="both", expand=1)
        hand_frame.pack(fill="both", expand=1)
        thresh_frame.pack(fill="both", expand=1)
        new_data_frame.pack(fill="both", expand=1)
        
        #change gesture -> action tab
        #skin detection/calibration tab
        
        
        tabs.add(go_live_frame, text="Go Live")
        tabs.add(thresh_frame, text="Thresholding")
        tabs.add(hand_frame, text="Mouse Control")
        tabs.add(cnn_frame, text="Classification")
        tabs.add(new_data_frame, text= "Add new data")
        tabs.bind("<<NotebookTabChanged>>", self.tab_change)
        
        
    def tab_change(self, event):
        selected_tab = event.widget.select()
        #print(selected_tab)
        if selected_tab == ".!frame":
            self.home_frame = True
            self.th_frame = False
            self.ha_frame = False
            self.class_frame = False
            self.add_data_frame = False
            self.normal_image()
            #print(0)
        elif selected_tab == ".!frame2":
            self.home_frame = False
            self.th_frame = True
            self.ha_frame = False
            self.class_frame = False
            self.add_data_frame = False
            self.show_thresh()
            #print(1)
        elif selected_tab == ".!frame4":
            self.home_frame = False
            self.th_frame = False
            self.ha_frame = True
            self.class_frame = False
            self.add_data_frame = False
            self.show_hands()
            #print(2)
        elif selected_tab == ".!frame3":
            self.add_data_frame = False
            self.home_frame = False
            self.th_frame = False
            self.ha_frame = False
            self.class_frame = True
            self.show_cnn()
            #print(3)
            
        elif selected_tab ==".!frame5":
            self.add_data_frame = True
            self.home_frame = False
            self.th_frame = False
            self.ha_frame = False
            self.class_frame = False
            self.add_data()
        else:
            print(selected_tab)
        
        
    def go_live_frame(self, container):
        frame = Frame(container, width =1280, height = 720, bg="blue")
        self.imghold = ttk.Label(frame)
        self.imghold.pack()
        btn=Button(frame, text="Go Live", command=lambda:  self.go_live(self.master))
        btn.pack()
        return frame
    
    def thresh_frame(self, container):
        frame = Frame(container, width =500, height = 500, bg="blue")
        self.thr_hold = ttk.Label(frame)
        self.thr_hold.pack()
        btn=Button(frame, text="Swap Thresholding Technique", command= lambda: print("1"))
        btn.pack()
        return frame
    
    def cnn_frame(self, container):
        frame = Frame(container, width=500, height=500, bg="red")
        self.cnn_hold = ttk.Label(frame)
        self.cnn_hold.pack()
        self.hand_pred= ttk.Label(frame)
        self.hand_pred.pack()
        self.hand_conf = ttk.Label(frame)
        self.hand_conf.pack()
        return frame
    
    def add_data_frame(self, container):
        frame = Frame(container, width=500, height=500, bg="orange")
        self.add_data_img = ttk.Label(frame)
        self.add_data_img.pack()
        leftbtn = Button(frame, text="Left Hand Toggle", command= self.toggle_left) 
        rightbtn = Button(frame, text="Right Hand Toggle", command= self.toggle_right) 
        #drop down menu
        
        self.clicked = tk.StringVar()
        self.clicked.set(self.class_names[0])
        
        drop = tk.OptionMenu(frame, self.clicked, *self.class_names)
        self.add_img = ttk.Label(frame)
        btn=Button(frame, text="Take Photos", command= lambda: self.threader_button(self.clicked.get()))
        
        drop.pack(pady= 5, side = tk.TOP)
        self.add_img.pack(side = tk.BOTTOM)
        leftbtn.pack(padx= 30, pady= 5, side = tk.LEFT)

        btn.pack(padx= 30, pady= 5,side = tk.RIGHT)
        rightbtn.pack(padx= 30, pady= 5,side = tk.LEFT)
        return frame
        
        
    def hand_frame(self, container):
        frame = Frame(container, width=500, height=500, bg="green")
        self.hand_hold= ttk.Label(frame)
        self.hand_hold.pack()
        btn=Button(frame, text="Press Enter", command= lambda: print("3"))
        btn.pack()
        return frame

    def go_live(self, container):
        print("Going Live")
        container.wm_state('iconic')
                       
        
# -------------- Tab Builders ----------------------------------------------------------------------
    def get_image(self):
        _, frame = self.capture.read()
        frame = cv2.flip(frame,1)
        #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA) #looks better on tkinter but scuffs pipeline
        return frame
    
    
    def show_image(self):
        frame = self.get_image()
        image = ImageTk.PhotoImage(image=PIL.Image.fromarray(frame))
        self.imghold.image = image
        self.imghold.configure(image=image)
        self.imghold.after(10, self.show_image)
    
    def normal_image(self):
        if self.home_frame == True:
            frame = self.get_image()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
            image = ImageTk.PhotoImage(image=PIL.Image.fromarray(frame))
            self.imghold.image = image
            self.imghold.configure(image=image)
            self.imghold.after(10, self.normal_image)


        
    def show_thresh(self):
        if self.th_frame == True:
            frame = self.get_image()
            thresh = self.imPRO.image_thresholder8(frame) #2 or 8 optimal
            image = ImageTk.PhotoImage(image=PIL.Image.fromarray(thresh))
            self.thr_hold.image = image
            self.thr_hold.configure(image=image)
            self.thr_hold.after(10, self.show_thresh)

            
    def show_hands(self):
        if self.ha_frame == True:
            frame = self.get_image()
            handcent = frame.copy()
            
            threshed = self.imPRO.image_thresholder8(handcent)
            biggestcont = self.imPRO.use_contours(threshed)
            biggestcont = biggestcont.astype(np.uint8)
            hand_cent_X, hand_cent_Y = self.imPRO.get_hand_centre(biggestcont)
            
            h, w = self.imPRO.get_image_centre(frame)
            r1, r2, r3, r4 = 75, 140, 200, 300
            self.imPRO.draw_guidelines(frame, r1,r2,r3,r4, h, w)
            cv2.circle(frame, (hand_cent_X, hand_cent_Y), 7, (255,255,0), -1)
            
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
            
            image = ImageTk.PhotoImage(image=PIL.Image.fromarray(frame))
            self.hand_hold.image = image
            self.hand_hold.configure(image=image)
            self.hand_hold.after(10, self.show_hands)
            

    def show_cnn(self):
        if self.class_frame == True:
            class_names = self.class_names
            frame = self.get_image()
            
            roi = self.imPRO.set_roi(frame, 0,0, 240,200) #now i need to get region of interest to follow centre of the hand.
            
            
            threshed = self.imPRO.image_thresholder8(roi)
            biggestcont = self.imPRO.use_contours(threshed)
            
            cnn_input1 = threshed.copy()
            biggestcont = biggestcont.astype(np.uint8)
            hand_cent_X, hand_cent_Y = self.imPRO.get_hand_centre(biggestcont)
            
            cnn_input = cnn_input1.copy()
            cnn_input = np.expand_dims(cnn_input, axis=0)
            cnn_input = np.expand_dims(cnn_input, axis=-1) #binary images have need to be added back
            
            predictions=self.model.predict(cnn_input)
            score = tf.nn.softmax(predictions[0])
            value=np.argmax(predictions)
            #print("This image most likely belongs to {} with a {:.2f} percent confidence.".format(class_names[np.argmax(score)], 100 * np.max(score)))
           
            
            image = ImageTk.PhotoImage(image=PIL.Image.fromarray(threshed))
            self.hand_pred.config(text="This image most likely belongs to {}".format(class_names[np.argmax(score)]))
            self.cnn_hold.image = image
            self.cnn_hold.configure(image=image)
            self.cnn_hold.after(10, self.show_cnn)
            self.hand_conf.config(text="with a {:.2f} confidence".format(100 * np.max(score)))
            
    #def show_cnn(self):
        #if self.class_frame == True:
            #class_names = ['rock', 'call_me', 'rock_on', 'fingers_crossed', 'peace', 'paper', 'okay', 'up', 'scissor', 'thumbs']
            #frame = self.get_image()          
            #
            #threshed = self.imPRO.image_thresholder8(frame)
            #biggestcont = self.imPRO.use_contours(threshed)
            
            
            #biggestcont = biggestcont.astype(np.uint8)
            #hand_cent_X, hand_cent_Y = self.imPRO.get_hand_centre(biggestcont)
            ##cv2.rectangle(frame, (hand_cent_X-120, hand_cent_Y-100),(hand_cent_X+120, hand_cent_Y+100), (255,255,0), 2)
            #cv2.rectangle(frame, (hand_cent_X-220, hand_cent_Y-220),(hand_cent_X+220, hand_cent_Y+220), (255,255,0), 2)
            
            ##roi = self.imPRO.set_roi(backcont, hand_cent_X,hand_cent_Y, hand_cent_X+200,hand_cent_Y+240) #now i need to get region of interest to follow centre of the hand.
            ##roi = np.expand_dims(roi, axis=0)
            ##roi = np.expand_dims(roi, axis=-1) #binary images have need to be added back
            
            ##predictions=self.model.predict(roi)
            ##score = tf.nn.softmax(predictions[0])
            ##value=np.argmax(predictions)
            ##print("This image most likely belongs to {} with a {:.2f} percent confidence.".format(class_names[np.argmax(score)], 100 * np.max(score)))
           
            
            #image = ImageTk.PhotoImage(image=PIL.Image.fromarray(frame))
            ##self.hand_pred.config(text="This image most likely belongs to {}".format(class_names[np.argmax(score)]))
            ##self.hand_conf.config(text="With a {:.2f} percent confidence.".format(100 * np.max(score)))
            #self.cnn_hold.image = image
            #self.cnn_hold.configure(image=image)
            #self.cnn_hold.after(10, self.show_cnn)
            
# ----------- Add Data Tab ---------------

    def get_class(self):
        return self.selected_class
        
    def set_class(self, new_class):
        self.selected_class = new_class
        

    def add_data(self):
        if self.add_data_frame == True:
            frame = self.get_image()
            thresh = self.imPRO.image_thresholder8(frame) #2 or 8 optimal
            h, w, c = frame.shape
            if self.add_left == True:
                cv2.rectangle(thresh, (0,0), (486, 406), (255,0,0), 3)
                self.roi = thresh[3:403, 3:483]
            if self.add_right == True:
                cv2.rectangle(thresh, (w-487,0), (w-1, 406), (255,255,0), 3)
                self.roi = thresh[3:403:, w-483:w-3]
            image = ImageTk.PhotoImage(image=PIL.Image.fromarray(thresh))
            self.add_data_img.image = image
            self.add_data_img.configure(image=image)
            self.add_data_img.after(10, self.add_data)
            
            #flicks too quickly, either slow it down or cap it. But it does work
            im = self.load_example()
            self.add_img.image = im
            self.add_img.configure(image=im)
            
            
    def toggle_left(self):
        self.add_central = False
        self.add_left = True
        self.add_right = False
        
    def toggle_right(self):
        self.add_central = False
        self.add_left = False
        self.add_right = True
               
    def threader_button(self, class_name):          
        if self.add_left != True and self.add_right != True:
            messagebox.showwarning("Warning", "Please select a position")
            return
        
                        
        #print(threading.active_count())
        t = threading.Thread(target=self.record_section, args = [class_name])
        t.daemon = True # This thread dies when main thread (only non-daemon thread) exits.
        t.start()
  
        
    def record_section(self, class_name):
        dt = datetime.datetime.now()
        if self.add_left == True:
            hand = "left"
        if self.add_right == True:
            hand = "right"
        i = 0
        
        #check/create folders
        self.new_image_dir()
        while True:
            time_delta = datetime.datetime.now() - dt
            if time_delta.total_seconds() >=10:
                #print("Done")
                break
            else:
                cv2.imwrite(f'new_images/{class_name}/{hand}_hand{i}.jpg', self.roi)
                #time.sleep(0.1) #1/30 fps
                time.sleep(1)
                i+=1
        return #ends a thread
    
    
    def new_image_dir(self):
        image_path = self.path + "/" + "new_images"
        if not os.path.exists(image_path):
            os.makedirs(image_path)
        for class_name in self.class_names:
            if not os.path.exists(image_path+"/"+class_name):
                os.makedirs(image_path+"/"+class_name)
                
    def load_example(self):
        class_name = self.clicked.get()       
        image_path = self.path + "\\" + "images" + "\\" + class_name + "\\"
        rand = random.choice(os.listdir(image_path))
        im = Image.open(image_path + rand)
        im = im.resize((100,100), Image.ANTIALIAS)
        
        im = ImageTk.PhotoImage(im)
        return im
        
            
    def image_countdown(self):
        print(3)
        time.sleep(2)
        print(2)
        time.sleep(2)
        print(1)
        time.sleep(2)
        

        
    

root = Tk()
my_gui = Hand_GUI(root)
root.mainloop()
my_gui.capture.release()